# Abgleich Angebot Fahrplan GTFS mit Zensus Daten im Gitterzellenraster (100m)

In [1]:
import duckdb
import datetime as dt
import pandas as pd

In [2]:
import locale
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')

'en_GB.UTF-8'

## initialisierung der Datenbank

In [6]:
duck = duckdb.connect(database=':memory:', read_only=False)

In [7]:
duck.sql("""
        install spatial;
        load spatial;
        INSTALL zipfs FROM community;
        LOAD zipfs;
        install httpfs;
        load httpfs;""")

## Laden der Gemeindegrenzen und Landkreise

In [8]:
duck.sql("""create or replace table gemeinden as
         select * from st_read('grenzen/VG250_GEM.shp')""")
duck.sql("""create or replace table landkreise as
         select * from st_read('grenzen/VG250_KRS.shp')""")

## Laden der Zensus-Daten im 100m-Raster

In [9]:
duck.sql("""create or replace table zensus_100m as
         select * 
         from read_csv('zip://zensus/Alter_in_5_Altersklassen.zip/Alter_in_5_Altersklassen/Zensus2022_Alter_in_5_Altersklassen_100m-Gitter.csv', 
         nullstr='–',
         sep=';')""")

In [12]:
duck.sql("from zensus_100m limit 5")

┌────────────────────────────────┬───────────┬───────────┬────────────────────────┬─────────┬──────────┬──────────┬──────────┬──────────────┐
│         GITTER_ID_100m         │ x_mp_100m │ y_mp_100m │ Insgesamt_Bevoelkerung │ Unter18 │ a18bis29 │ a30bis49 │ a50bis64 │ a65undaelter │
│            varchar             │   int64   │   int64   │         int64          │  int64  │  int64   │  int64   │  int64   │    int64     │
├────────────────────────────────┼───────────┼───────────┼────────────────────────┼─────────┼──────────┼──────────┼──────────┼──────────────┤
│ CRS3035RES100mN2689100E4337000 │   4337050 │   2689150 │                      4 │    NULL │     NULL │     NULL │     NULL │         NULL │
│ CRS3035RES100mN2689100E4341100 │   4341150 │   2689150 │                     11 │    NULL │     NULL │     NULL │     NULL │         NULL │
│ CRS3035RES100mN2690800E4341200 │   4341250 │   2690850 │                      4 │    NULL │     NULL │     NULL │     NULL │         NULL │
│ CRS3

## Laden GTFS aus zip-File

In [13]:
trips = 'zip://gtfs/gtfs_top_level.zip/trips.txt'
routes = 'zip://gtfs/gtfs_top_level.zip/routes.txt'
agency = 'zip://gtfs/gtfs_top_level.zip/agency.txt'
calendar = 'zip://gtfs/gtfs_top_level.zip/calendar.txt'
calendar_dates = 'zip://gtfs/gtfs_top_level.zip/calendar_dates.txt'
stops = 'zip://gtfs/gtfs_top_level.zip/stops.txt'
stop_times = 'zip://gtfs/gtfs_top_level.zip/stop_times.txt'
frequencies = 'zip://gtfs/gtfs_top_level.zip/frequencies.txt'
transfers = 'zip://gtfs/gtfs_top_level.zip/transfers.txt'
service_alerts = 'zip://gtfs/gtfs_top_level.zip/service_alerts.txt'
shapes = 'zip://gtfs/gtfs_top_level.zip/shapes.txt'

In [14]:
def gtfs_init_sql():
    sql = """
    install spatial;
    load spatial;
    INSTALL zipfs FROM community;
    LOAD zipfs;
    install httpfs;
    load httpfs;
    --Einlesen der Linientabelle aus DM
    -- erfolgt jetzt über attach 02.12.2024
    --create or replace table linien_dm as select * from read_csv_auto('input/linien_dm.csv');

    --Erzeugen eines ENUM Type (begrenzte Anzahl von Werten) nur mit String nicht integer
    --bei Type gibt es kein replace, daher können diese nur einmalig ausgeführt, ansonsten Error
    CREATE TYPE enum01 AS ENUM ('0', '1');
    CREATE TYPE enum012 AS ENUM ('0', '1', '2');
    CREATE TYPE enum0123 AS ENUM ('0', '1', '2', '3');
    CREATE TYPE enum01234 AS ENUM ('0', '1', '2', '3', '4');
    CREATE TYPE enum12 AS ENUM ('1', '2');
    CREATE TYPE enum_route_type AS ENUM ('0','1', '2','3', '4', '5', '6', '7', '11', '12');
        """
    return sql

In [15]:
def gtfs_load_sql(agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes):
    """
    Generates SQL to create GTFS tables in DuckDB.
    Parameters:
    - agency: Path to the agency.txt file.
    - calendar: Path to the calendar.txt file.
    - calendar_dates: Path to the calendar_dates.txt file.
    - routes: Path to the routes.txt file.
    - trips: Path to the trips.txt file.
    - stops: Path to the stops.txt file.
    - stop_times: Path to the stop_times.txt file.
    - frequencies: Path to the frequencies.txt file.
    - service_alerts: Path to the service_alerts.txt file (default is set).
    Returns:
    - SQL string to create the GTFS tables.
    """

    sql = f"""
    -- Skript zum Erzeugen einer DuckDB Datenbank mit GTFS Tabellen
    -- Version für Toplevel
    -- Stand 21.07.2025 mit Festlegen der Datenformate
    -- Tausch der Spaltenreihenfolge bei Stop_times

    load spatial;
    load httpfs;
    load zipfs;

    --Einlesen der GTFS Tabellen
    --create or replace table agency as select * from read_csv_auto('/home/ts/python/duckdb/gtfs/agency.txt');
    CREATE or REPLACE table agency AS 
        select * from read_csv('{agency}', delim=',', header=true, dateformat = '%Y%m%d',
        columns={{'agency_id': 'VARCHAR', 
        'agency_name': 'VARCHAR', 
        'agency_url': 'VARCHAR', 
        'agency_timezone': 'VARCHAR', 
        'agency_lang': 'VARCHAR', 
        'agency_phone':'VARCHAR'}});

    --create or replace table calendar as select * from read_csv_auto('calendar.txt');
    CREATE OR REPLACE table calendar AS 
        select * from read_csv('{calendar}', delim=',', header=true, dateformat = '%Y%m%d',
        columns={{'service_id': 'VARCHAR', 
        'monday': 'enum01', 
        'tuesday': 'enum01', 
        'wednesday': 'enum01', 
        'thursday':'enum01', 
        'friday':'enum01',
        'saturday': 'enum01',
        'sunday' : 'enum01',
        'start_date' : 'DATE',
        'end_date' : 'DATE'}});
    --create or replace table calendar as select * from read_csv_auto('/home/ts/python/duckdb/gtfs/calendar.txt');

    --ALTER TABLE calendar ALTER monday TYPE smallint;
    --create or replace table calendar_dates as select * from read_csv_auto('/home/ts/python/duckdb/gtfs/calendar_dates.txt');
    CREATE or REPLACE table calendar_dates AS 
        select * from read_csv('{calendar_dates}', delim=',', header=true, dateformat = '%Y%m%d',
        columns={{'service_id': 'VARCHAR', 
        'date': 'DATE', 
        'exception_type': 'enum12'}});

    create or replace table frequencies as select * from read_csv_auto('{frequencies}');
    -- Levels / Pathways in TOP nicht enthalten
    --create or replace table levels as select * from read_csv_auto('gtfs_top/levels.txt');
    --create or replace table pathways as select * from read_csv_auto('gtfs_top/pathways.txt');
    --create or replace table routes as select * from read_csv_auto('/home/ts/python/duckdb/gtfs/routes.txt');
    CREATE or REPLACE table routes AS 
        select * from read_csv('{routes}', delim=',', header=true, dateformat = '%Y%m%d',
        columns={{'route_id': 'VARCHAR', 
        'agency_id': 'VARCHAR', 
        'route_short_name': 'VARCHAR', 
        'route_long_name': 'VARCHAR', 
        'route_type':'INTEGER',  -- eigentlich enum aber fehlerhafte Daten mit route_type 715
        'route_color':'VARCHAR',
        'route_text_color': 'VARCHAR',
        'route_desc' : 'VARCHAR'}});

    create or replace table service_alerts as select * from read_csv_auto('{service_alerts}');
    create or replace table shapes as select * from read_csv_auto('{shapes}');
    --create or replace table stop_times as select * from read_csv_auto('/home/ts/python/duckdb/gtfs/stop_times.txt');
    CREATE or REPLACE table stop_times AS 
        select * from read_csv(
            '{stop_times}', 
            delim=',', 
            header=true, 
            dateformat = '%Y%m%d', 		
            ignore_errors= true,
        columns={{
        'trip_id': 'VARCHAR', 
        'stop_id':'VARCHAR', 
        'stop_sequence':'INT16',	
        'pickup_type' : 'VARCHAR',
        'drop_off_type' : 'VARCHAR',
        'stop_headsign': 'VARCHAR',
        'arrival_time': 'VARCHAR', 
        'departure_time': 'VARCHAR'
        }},  store_rejects = true
        );

    CREATE or REPLACE table stops AS 
        select * from read_csv('{stops}', delim=',', header=true, dateformat = '%Y%m%d',
        columns={{'stop_id': 'VARCHAR', 
        'stop_code': 'VARCHAR', 
        'stop_name': 'VARCHAR', 
        'stop_desc': 'VARCHAR', 
        'stop_lat':'DOUBLE', 
        'stop_lon':'DOUBLE',	
        'location_type' : 'enum01234',
        'parent_station' : 'VARCHAR',
        'wheelchair_boarding': 'enum012',
        'platform_code': 'VARCHAR',
        'zone_id': 'VARCHAR'
        }},
        store_rejects = true);

    create or replace table transfers as select * from read_csv_auto('{transfers}');
 
    CREATE or REPLACE TABLE trips AS 
        select * from read_csv('{trips}', delim=',', header=true, dateformat = '%Y%m%d',
        columns={{
            'route_id' : 'VARCHAR', 
            'service_id': 'VARCHAR', 
        'trip_id': 'VARCHAR', 
        'trip_headsign': 'VARCHAR', 
        'trip_short_name': 'VARCHAR', 
        'direction_id':'enum01', 
        'block_id':'VARCHAR',
        'shape_id': 'VARCHAR',
        'wheelchair_accessible' : 'enum012',
        'bikes_allowed' : 'enum012'
        }},
        store_rejects = true);

    --Einlesen der VBN Grenzen
    create or replace table vbn as select * from st_read('grenzen/vbn.gpkg');

    --Einlesen HIS und Erstellen einer Geometry-Spalte
    create or replace table his_akt as select * from "https://daten.zvbn.de/his_akt.csv";
    alter table his_akt add column geom Geometry;
    UPDATE his_akt set geom = st_point(x_wgs, y_wgs);

    --Erstellen einer Tabelle mit Geom-Spalte
    create or replace table stops_geom as select *, st_point(stop_lon, stop_lat) as geom from stops;
    ALTER TABLE stops add column geom Geometry;
    UPDATE stops set geom = st_point(stop_lon, stop_lat);

    -- Erstellen einer Tabelle Stops im VBN
    create or replace table stops_vbn as 
        select s.stop_id, s.stop_name, s.geom from stops s join vbn on st_within(s.geom, vbn.geom);

    -- Verknüpfen der Stops mit den Trips und den Routen für die Routen im VBN
    create or replace view vw_routes_vbn as 
        select distinct r.route_id, r.route_type, r.route_short_name, a.agency_id, a.agency_name from
        stop_times st 
        join stops_vbn s_vbn on s_vbn.stop_id = st.stop_id
        join trips t on st.trip_id = t.trip_id
        join routes r on t.route_id = r.route_id
        join agency a on a.agency_id = r.agency_id;

    -- Verknüpfen der Stops mit den Trips und den Routen für die Routen im VBN > feste Tabelle
    create or replace table tbl_routes_vbn as 
        select distinct r.route_id, r.route_type, r.route_short_name, a.agency_id, a.agency_name from
        stop_times st 
        join stops_vbn s_vbn on s_vbn.stop_id = st.stop_id
        join trips t on st.trip_id = t.trip_id
        join routes r on t.route_id = r.route_id
        join agency a on a.agency_id = r.agency_id;

    -- Verknüpfung des Verlaufs mit den Haltestellennamen
    create or replace view vw_trip_stop as
        SELECT a.agency_id, r.route_id, r.route_short_name, st.trip_id, t.trip_short_name, t.service_id, st.stop_sequence, st.stop_id, s.stop_name, st.arrival_time, st.departure_time  
        FROM stop_times st 
        JOIN stops s on s.stop_id = st.stop_id 
        JOIN trips t on st.trip_id = t.trip_id 
        JOIN routes r on r.route_id = t.route_id
        JOIN agency a on a.agency_id = r.agency_id;

    """

    return sql

In [16]:
print(gtfs_load_sql(agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes))


    -- Skript zum Erzeugen einer DuckDB Datenbank mit GTFS Tabellen
    -- Version für Toplevel
    -- Stand 21.07.2025 mit Festlegen der Datenformate
    -- Tausch der Spaltenreihenfolge bei Stop_times

    load spatial;
    load httpfs;
    load zipfs;

    --Einlesen der GTFS Tabellen
    --create or replace table agency as select * from read_csv_auto('/home/ts/python/duckdb/gtfs/agency.txt');
    CREATE or REPLACE table agency AS 
        select * from read_csv('zip://gtfs/gtfs_top_level.zip/agency.txt', delim=',', header=true, dateformat = '%Y%m%d',
        columns={'agency_id': 'VARCHAR', 
        'agency_name': 'VARCHAR', 
        'agency_url': 'VARCHAR', 
        'agency_timezone': 'VARCHAR', 
        'agency_lang': 'VARCHAR', 
        'agency_phone':'VARCHAR'});

    --create or replace table calendar as select * from read_csv_auto('calendar.txt');
    CREATE OR REPLACE table calendar AS 
        select * from read_csv('zip://gtfs/gtfs_top_level.zip/calendar.txt', delim=',', 

In [17]:
duck.sql(gtfs_init_sql())

In [18]:
duck.sql(gtfs_load_sql(agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes))

In [19]:
duck.sql("show all tables")

┌──────────┬─────────┬────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Zensus Daten im VBN
- st_dwithin bringt nochmal Beschleunigung
- https://duckdb.org/2025/08/08/spatial-joins.html

In [ ]:
duck.sql("""create or replace table vbn_zensus as 
         select z.* exclude (x_mp_100m, y_mp_100m),  st_transform(st_point(z.x_mp_100m, z.y_mp_100m), 'EPSG:3035', 'EPSG:4326', always_xy := true) as geom
         from zensus_100m z, vbn
         where st_dwithin(
            st_transform(st_point(z.x_mp_100m, z.y_mp_100m), 'EPSG:3035', 'EPSG:4326', always_xy := true), 
            vbn.geom, 
            0
         )
         -- limit 10;
         """)

In [23]:
duck.sql("from vbn_zensus")

┌────────────────────────────────┬────────────────────────┬─────────┬──────────┬──────────┬──────────┬──────────────┬──────────────────────────────────────────────┐
│         GITTER_ID_100m         │ Insgesamt_Bevoelkerung │ Unter18 │ a18bis29 │ a30bis49 │ a50bis64 │ a65undaelter │                     geom                     │
│            varchar             │         int64          │  int64  │  int64   │  int64   │  int64   │    int64     │                   geometry                   │
├────────────────────────────────┼────────────────────────┼─────────┼──────────┼──────────┼──────────┼──────────────┼──────────────────────────────────────────────┤
│ CRS3035RES100mN3260700E4208800 │                     35 │       3 │        8 │        8 │       15 │            9 │ POINT (8.35048789988157 52.444672564478445)  │
│ CRS3035RES100mN3260700E4208900 │                     30 │       4 │        3 │        8 │        6 │            7 │ POINT (8.351958241687827 52.44469292104891)  │
│ CRS3035R

In [ ]:
duck.sql("""select sum(Insgesamt_Bevoelkerung) as sum_ew,
     sum(unter18) as sum_unter18,
        sum(a18bis29) as sum_a18bis29,
         sum(a30bis49) as sum_a30bis49,
         sum(a50bis64) as sum_a50bis64,
         sum(a65undaelter) as sum_a65undaelter
    from vbn_zensus
    """)

## Abgleich mit GTFS

### Ermitteln der Einwohner im Einzugsbereich aller Haltestellen

In [ ]:
radius = 1000  # Radius in meters
duck.sql(f"""select 
         sum(Insgesamt_Bevoelkerung) as sum_ew,
     sum(unter18) as sum_unter18,
        sum(a18bis29) as sum_a18bis29,
         sum(a30bis49) as sum_a30bis49,
         sum(a50bis64) as sum_a50bis64,
         sum(a65undaelter) as sum_a65undaelter from
         (select distinct z.*
         from vbn_zensus z, 
         
         stops s
         
            where st_within(
            st_transform(z.geom, 'EPSG:4326', 'EPSG:25832'),    
            st_buffer(
                st_transform(s.geom, 'EPSG:4326', 'EPSG:25832')
            , {radius} )
         )
         group by all)
         """)

### Ermitteln der Haltestellen die an einem bestimmten Tag angefahren werden und ermitteln der Einwohner

In [ ]:
locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8') # damit Wochentag in englisch
# Example string
date_string = "2025-08-03"
radius = 1000  # Radius in meters
mindest_abfahrten = 0  # Mindestanzahl an Abfahrten

# Convert string to date
date_object = dt.datetime.strptime(date_string, "%Y-%m-%d").date()
weekday_today_name = date_object.strftime('%A').lower()

# Ermitteln der Service_id an dem Verkehrstag
duck.sql(f"""create or replace table verkehrt_{date_object.strftime('%Y%m%d')} as 
         select *, true as verkehrt from
         (select distinct t.service_id, cd.verkehrt_cd, c.verkehrt_c, 
         from trips t
         left join (select service_id, date, exception_type as verkehrt_cd from calendar_dates 
         where date =  '{date_object.strftime('%Y-%m-%d')}' and exception_type = 1) as cd 
         on t.service_id = cd.service_id
         left join  (select service_id, {date_object.strftime('%A').lower()} as verkehrt_c
         from calendar
         where {date_object.strftime('%A').lower()} = 1) as c
         on t.service_id = c.service_id
         where c.verkehrt_c = 1 or cd.verkehrt_cd = 1)""")

#Ermitteln der Trips an dem Verkehrstag
duck.sql(f"""create or replace table trips_{date_object.strftime('%Y%m%d')} as
         select * 
         from trips t
         join (select service_id 
            from  verkehrt_{date_object.strftime('%Y%m%d')} 
            where verkehrt_cd = 1 or verkehrt_c = 1) vh on t.service_id = vh.service_id
         join vw_routes_vbn r on t.route_id = r.route_id
         """)

#Ermitteln der Haltestellen an dem Verkehrstag
duck.sql(f"""select distinct s.* 
         from trips_{date_object.strftime('%Y%m%d')} ta, 
         trips t, stop_times st, stops s
         where ta.trip_id = t.trip_id and t.trip_id = st.trip_id and st.stop_id = s.stop_id
        """)

#Ermitteln der statistischen Werte
duck.sql(f"""select 
         sum(Insgesamt_Bevoelkerung) as sum_ew,
        sum(unter18) as sum_unter18,
        sum(a18bis29) as sum_a18bis29,
         sum(a30bis49) as sum_a30bis49,
         sum(a50bis64) as sum_a50bis64,
         sum(a65undaelter) as sum_a65undaelter from
         (select distinct z.*
         from vbn_zensus z, 
         
         (
        select distinct s.stop_id, s.stop_name, s.geom, s.stop_lon, s.stop_lat, count(t.trip_id) as anzahl_abfahrten
         from trips_{date_object.strftime('%Y%m%d')} ta, 
         trips t, stop_times st, stops s
         where ta.trip_id = t.trip_id and t.trip_id = st.trip_id and st.stop_id = s.stop_id 
         group by all
         ) s
         
            where st_within(
            st_transform(z.geom, 'EPSG:4326', 'EPSG:25832'),    
            st_buffer(
                st_transform(s.geom, 'EPSG:4326', 'EPSG:25832')
            , {radius} )
           
         ) and s.anzahl_abfahrten > {mindest_abfahrten}  -- Mindestanzahl an Abfahrten
         group by all)
         """)

### Ermitteln der Abfahrten je Haltestelle

In [ ]:
duck.sql(f"""
select distinct s.stop_id, s.stop_name, s.geom, s.stop_lon, s.stop_lat, count(t.trip_id) as anzahl_abfahrten
         from trips_{date_object.strftime('%Y%m%d')} ta, 
         trips t, stop_times st, stops s
         where ta.trip_id = t.trip_id and t.trip_id = st.trip_id and st.stop_id = s.stop_id 
         group by all         

""")

#### Ermitteln der erschlossenen Einwohnern je Gemeinde an einem Verkehrstag

In [ ]:
duck.sql(f"""create or replace table zensus_vt_{date_object.strftime('%Y%m%d')} as
         select ags, gen, 
         sum(Insgesamt_Bevoelkerung) as sum_ew,
        sum(unter18) as sum_unter18,
        sum(a18bis29) as sum_a18bis29,
         sum(a30bis49) as sum_a30bis49,
         sum(a50bis64) as sum_a50bis64,
         sum(a65undaelter) as sum_a65undaelter
         
         from
         (
        select distinct z.*
         from vbn_zensus z, 
         
         (
         select distinct s.stop_id, s.stop_name, s.geom, s.stop_lon, s.stop_lat, count(t.trip_id) as anzahl_abfahrten
         from trips_{date_object.strftime('%Y%m%d')} ta, 
         trips t, stop_times st, stops s
         where ta.trip_id = t.trip_id and t.trip_id = st.trip_id and st.stop_id = s.stop_id
         group by all) s
         
            where st_within(
            st_transform(z.geom, 'EPSG:4326', 'EPSG:25832'),    
            st_buffer(
                st_transform(s.geom, 'EPSG:4326', 'EPSG:25832')
            , {radius} ))
            and s.anzahl_abfahrten > {mindest_abfahrten}  -- Mindestanzahl an Abfahrten
            ) as ew_vt,
             gemeinden g
            where st_within(st_transform(ew_vt.geom,'EPSG:4326' ,'EPSG:25832', always_xy := true), g.geom )
            group by all
            order by ags
               """)
duck.sql(f"from zensus_vt_{date_object.strftime('%Y%m%d')}").df()

### Ermitteln aller Einwohner

In [ ]:
duck.sql(f"""create or replace table zensus_gesamt as
         select ags, gen, 
         sum(Insgesamt_Bevoelkerung) as sum_ew_ges,
        sum(unter18) as sum_unter18_ges,
        sum(a18bis29) as sum_a18bis29_ges,
         sum(a30bis49) as sum_a30bis49_ges,
         sum(a50bis64) as sum_a50bis64_ges,
         sum(a65undaelter) as sum_a65undaelter_ges
         
         from vbn_zensus z,
         gemeinden g
         where st_within(st_transform(z.geom,'EPSG:4326' ,'EPSG:25832', always_xy := true), g.geom )
         group by all
         order by ags
         """)

### Ermitteln des Anteils der Einwohner je Gemeinde (Mitgliegsgemeinde)

In [ ]:
duck.sql(f"""select g.ags, g.gen, g.sum_ew_ges, v.sum_ew, round(v.sum_ew/g.sum_ew_ges, 3) as anteil_ew_vt
         from zensus_gesamt g
         left join zensus_vt_{date_object.strftime('%Y%m%d')} v on g.ags = v.ags
         
         order by ags""").df()

## Abbildung als Klasse

In [ ]:
class GTFSZensusDB:
    def __init__(self, 
                 agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes):
        import duckdb
        self.conn = duckdb.connect(database=':memory:', read_only=False)
        # Initialize GTFS types
        self.conn.sql(gtfs_init_sql())
        # Load GTFS tables
        self.conn.sql(gtfs_load_sql(agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes))
        #Laden der VBN Grenzen, Gemeinden und Landkreise
        self.conn.sql("""create or replace table gemeinden as
            select * from st_read('grenzen/VG250_GEM.shp')""")
        self.conn.sql("""create or replace table landkreise as
             select * from st_read('grenzen/VG250_KRS.shp')""")
        # Create a table for the 100m grid census data
        self.conn.sql("""create or replace table zensus_100m as
         select * 
         from read_csv('zip://zensus/Alter_in_5_Altersklassen.zip/Alter_in_5_Altersklassen/Zensus2022_Alter_in_5_Altersklassen_100m-Gitter.csv', 
         nullstr='–',
         sep=';')""")
        
    def query(self, sql):
        return self.conn.sql(sql).df()

    def get_stop_departure_counts(self, date_object):
        return self.conn.sql(f"""
        select distinct s.stop_id, s.stop_name, s.geom, s.stop_lon, s.stop_lat, count(t.trip_id) as anzahl_abfahrten
         from trips_{date_object.strftime('%Y%m%d')} ta, 
         trips t, stop_times st, stops s
         where ta.trip_id = t.trip_id and t.trip_id = st.trip_id and st.stop_id = s.stop_id 
         group by all         

""")

# Example usage:
# db = GTFSZensusDB(agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes)
# result = db.query('show tables')

In [ ]:
db = GTFSZensusDB(agency, calendar, calendar_dates, routes, trips, stops, stop_times, frequencies, transfers, service_alerts, shapes)
result = db.query('show tables')

In [ ]:
result